# Functional programming

## Functions are values

Functions are values, *almost* like any other values in OCaml.    They live in the same namespace, can be passed around, but cannot be tested for equality or serialized to disk.

`f` is a function which takes two arguments and returns their sum:

In [1]:
let f x y = x + y

val f : int -> int -> int = <fun>

We can pass it to `List.fold_left`, a *higher-order function*, to find the sum of a list of integers:

In [19]:
List.fold_left

- : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a = <fun>

In [20]:
List.fold_right

- : ('a -> 'b -> 'b) -> 'a list -> 'b -> 'b = <fun>

In [2]:
List.fold_left f 0 [1;2;3]

- : int = 6

`List.fold_left` is a generic function that doesn't know anything about the values it is dealing with.   We can just as easily use it to concatenate strings, by passing in a different function and starting value:

In [24]:
(+.)

- : float -> float -> float = <fun>

In [3]:
List.fold_left ( ^ ) "" ["foo"; "bar"; "baz"]

- : string = "foobarbaz"

We can also *compose* functions together, yielding new functions which work just as if they had been defined directly.   `compose` is just a function:

In [4]:
let compose g f x = g (f x)

val compose : ('a -> 'b) -> ('c -> 'a) -> 'c -> 'b = <fun>

In [5]:
let m x = x + 1
let n x = x * 2
let mn = compose m n

val m : int -> int = <fun>

val n : int -> int = <fun>

val mn : int -> int = <fun>

Here, we have defined two small functions `m` and `n`, then composed them to yield a new function `mn` which applies both of them to its argument:

In [25]:
mn 4;;
m (n 4)

- : int = 9

- : int = 9

We can pass `mn` around and use it just like any other function:

In [26]:
List.map mn [1;2;3]

- : int list = [3; 5; 7]

## Currying and partial application

The functions we have seen so far have been defined in *curried* style.   We could equally well have defined `f` like this (uncurried):

In [8]:
let f' (x, y) = x + y

val f' : int * int -> int = <fun>

In this case, `f'` expects a *tuple* containing two elements, `x` and `y`.   A tuple is like a `struct` with anonymous fields.

Most Ocaml code defines functions in the curried style, because it permits *partial application*.    If we call a curried function and pass fewer arguments than it expects, the result will be a *new* function which takes the remaining arguments and returns the result.

`add42` is a specialized function which only adds 42 to its argument:

In [9]:
let add42 = f 42

val add42 : int -> int = <fun>

In [10]:
List.map add42 [1;2;3]

- : int list = [43; 44; 45]

## Closures

When defining a function, you might refer to a binding in the enclosing scope - rather than just locals and function parameters.   That binding will be captured in a *closure* with your function, and will travel with it as you pass the function around.   This is true even if the captured binding was local to the enclosing scope:  

In [27]:
let outer_scope outer_param = 
   let local_binding = "local_binding" in
   let closure closure_param =
      Printf.printf "outer_param: %s, local_binding: %s, closure_param: %s\n" outer_param local_binding closure_param
   in closure

val outer_scope : string -> string -> unit = <fun>

In [28]:
let c = outer_scope "outer_param" in
c "closure_param"

outer_param: outer_param, local_binding: local_binding, closure_param: closure_param


- : unit = ()

Closures are extremely useful.   One simple use is to make object-like constructions which contain encapsulated state which can only be accessed through accessor functions:

In [13]:
let inc_count, dec_count, get_count =
    let counter = ref 0 in
    let _inc_count () = incr counter in
    let _dec_count () = decr counter in
    let _get_count () = !counter in
    _inc_count, _dec_count, _get_count

val inc_count : unit -> unit = <fun>
val dec_count : unit -> unit = <fun>
val get_count : unit -> int = <fun>

In [14]:
inc_count (); inc_count (); dec_count (); inc_count(); get_count ()

- : int = 2

## Immutable data structures

Most data structures in Ocaml are immutable, although as the previous example demonstrated, mutable values are available through `ref`.   Immutability simplifies reasoning about complex programs, and also makes testing easier - a *pure* function (one with no side effects) operating on an *immutable data structure* must give the same result each time it is executed.

The advantage of immutability is that data structures can't be modified, so reasoning about them is easier.   However mutability can also be a disadvantage, for instance when performance is critically important, or when an algorithm is most easily expressed with mutable data structures.   

Say we have a list, and we want to add an element at the beginning of it.   We can't change the list, so our only option is to return a new list containing all the elements of the old list, plus the new one.   This might seem really inefficient, but in fact immutability is critical in making this a fairly efficient operation.   We have to create a new *list*, but we don't have to copy all the *elements* of the old one - we can just refer to them from our new list.   Since the elements of the old list are immutable, there is no logical difference between this and copying them outright:  

In [15]:
let t = [3;2;1]
let t1 = 4 :: t
let t2 = 4 :: t

val t : int list = [3; 2; 1]

val t1 : int list = [4; 3; 2; 1]

val t2 : int list = [4; 3; 2; 1]

`t1` and `t2` contain the same values, which  Ocaml's logical equality operator confirms:

In [16]:
t1 = t2

- : bool = true

The physical equality (`==`) operator shows that the two lists are different in memory:

In [17]:
t1 == t2

- : bool = false

However this is just because the heads of the two lists differ - `t1` and `t2` each have their own, separately-allocated head elements, but they both point to the same tails:

In [18]:
List.tl t1 == List.tl t2

- : bool = true

If memory is tight, or we are allocating millions of logically identical nodes, we can avoid making multiple physical copies of the same logical data by [hash consing](http://en.wikipedia.org/wiki/Hash_consing).   This optimization would not be safe in general without immutable data.

If we want to add a value at the end of the list, we are stuck with re-creating the entire list.   This is why functional programs often use trees, even to store 'linear' data - in general, when a node is removed from a tree, the number of nodes which must be copied is proportional to the node's depth, not the size of the tree.